In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing
import json
from data_utils import *
from model import *
from algorithm import *
import random
import os
import torch.optim as optim

#### In this file, the decision-maker publishes h as the policy. So both the qualification change and score change are measured by h

### Hiring

In [ ]:
c_idx = [0,3]
nc_idx = [1,2]
h, f = logReg(5), logReg(5)
h.load_state_dict(torch.load('h_models/h_hiring_lr.pth'))
f.load_state_dict(torch.load('f_models/f_lr_hiring.pth'))

In [ ]:
h.state_dict()

In [ ]:
original = pd.read_csv('data/hiring_data_sample.csv')
original['z'] = original['age']
X_original = original[['education','YearsCode','PreviousSalary','ComputerSkills','z']]
data = Hiring(device='cpu', decision=False)
X_original = (X_original - data.mean)/data.std_dev
X_original = X_original.to_numpy()
X_original = torch.Tensor(X_original)

# simulate best response
We = torch.tensor([1,1,2,2], dtype = torch.float32)

efforts, x_star, x_improve = Grad_effort(h, X_original, We, c_idx, nc_idx)

In [ ]:
# sanity check
print(x_improve[0])
print(x_star[0])
print(X_original[0])
print(np.isnan(x_improve.detach().numpy()).any())
print(np.isnan(x_star.detach().numpy()).any())

In [ ]:
original_scores = np.round(h(X_original).detach().numpy(),3).reshape(-1)
improve_scores = np.round(h(x_improve).detach().numpy(),3).reshape(-1)
strategy_scores = np.round(h(x_star).detach().numpy(),3).reshape(-1)

print(original_scores.mean())
print(improve_scores.mean())
print(strategy_scores.mean())

In [ ]:
g = np.array(original['z'])

# save scores
np.save('results/hiring_original_scores.npy', original_scores)
np.save('results/hiring_improve_scores.npy', improve_scores)
np.save('results/hiring_strategy_scores.npy', strategy_scores)

# save efforts
np.save('results/hiring_efforts.npy', efforts.detach().numpy())

# save groupwise scores
np.save('results/hiring_original_scores_0.npy', original_scores[g==0])
np.save('results/hiring_improve_scores_0.npy', improve_scores[g==0])
np.save('results/hiring_strategy_scores_0.npy', strategy_scores[g==0])

np.save('results/hiring_original_scores_1.npy', original_scores[g==1])
np.save('results/hiring_improve_scores_1.npy', improve_scores[g==1])
np.save('results/hiring_strategy_scores_1.npy', strategy_scores[g==1])

# save groupwise efforts
np.save('results/hiring_efforts_0.npy', efforts.detach().numpy()[g==0])
np.save('results/hiring_efforts_1.npy', efforts.detach().numpy()[g==1])


### Law

In [ ]:
c_idx = [0,1]
nc_idx = []

In [ ]:
h, f = logReg(3), logReg(3)
h.load_state_dict(torch.load('h_models/h_law_lr.pth'))
f.load_state_dict(torch.load('f_models/f_lr_law.pth'))

In [ ]:
original = pd.read_csv('data/bar_pass_data_sample.csv')
original['z'] = original['sex']
X_original = original[['ugpa','lsat','z']]
data = LawDataset(device='cpu', decision=False)
X_original = (X_original - data.mean)/data.std_dev
X_original = X_original.to_numpy()
X_original = torch.Tensor(X_original)

# simulate best response
We = torch.tensor([0.5,0.5], dtype = torch.float32)

efforts, x_star, x_improve = Grad_effort(h, X_original, We, c_idx, nc_idx)

In [ ]:
# sanity check
print(x_improve[0])
print(x_star[0])
print(X_original[0])
print(np.isnan(x_improve.detach().numpy()).any())
print(np.isnan(x_star.detach().numpy()).any())

In [ ]:
original_scores = np.round(h(X_original).detach().numpy(),3).reshape(-1)
improve_scores = np.round(h(x_improve).detach().numpy(),3).reshape(-1)
strategy_scores = np.round(h(x_star).detach().numpy(),3).reshape(-1)

print(original_scores.mean())
print(improve_scores.mean())
print(strategy_scores.mean())

In [ ]:
g = np.array(original['z'])

# save scores
np.save('results/law_original_scores.npy', original_scores)
np.save('results/law_improve_scores.npy', improve_scores)
np.save('results/law_strategy_scores.npy', strategy_scores)

# save efforts
np.save('results/law_efforts.npy', efforts.detach().numpy())

# save groupwise scores
np.save('results/law_original_scores_0.npy', original_scores[g==1])
np.save('results/law_improve_scores_0.npy', improve_scores[g==1])
np.save('results/law_strategy_scores_0.npy', strategy_scores[g==1])

np.save('results/law_original_scores_1.npy', original_scores[g==2])
np.save('results/law_improve_scores_1.npy', improve_scores[g==2])
np.save('results/law_strategy_scores_1.npy', strategy_scores[g==2])

# save groupwise efforts
np.save('results/law_efforts_0.npy', efforts.detach().numpy()[g==1])
np.save('results/law_efforts_1.npy', efforts.detach().numpy()[g==2])


### Credit

- specify stategic indexes

In [ ]:
c_idx = [2,3]
nc_idx = [0,4,6]

- Get the decision models

In [ ]:
h, f = logReg(10), logReg(10)
h.load_state_dict(torch.load('h_models/h_credit_lr.pth'))
f.load_state_dict(torch.load('f_models/f_LR_credit.pth'))

- Simulate best responses

In [ ]:
data = CreditDataset(device = 'cpu', decision=False)
original = pd.read_csv('data/balanced_test_dataset.csv')
original['z'] = np.where(original['age'] > 35, 0, 1)
X_original = original.drop(columns = ['age','question', 'qualification_status'])
X_original = (X_original - data.mean)/data.std_dev
X_original = X_original.to_numpy()
X_original = torch.Tensor(X_original)

# simulate best response
We = torch.tensor([0.5,0.5,2,2,2], dtype = torch.float32)

efforts, x_star, x_improve = Grad_effort(h, X_original, We, c_idx, nc_idx)

- Sanity check

In [ ]:
# sanity check
print(x_improve[0])
print(x_star[0])
print(X_original[0])
print(np.isnan(x_improve.detach().numpy()).any())
print(np.isnan(x_star.detach().numpy()).any())

- Get score increase

In [ ]:
original_scores = np.round(h(X_original).detach().numpy(),3).reshape(-1)
improve_scores = np.round(h(x_improve).detach().numpy(),3).reshape(-1)
strategy_scores = np.round(h(x_star).detach().numpy(),3).reshape(-1)
print(original_scores.mean())
print(improve_scores.mean())
print(strategy_scores.mean())

- Save results

In [ ]:
g = np.array(original['z'])

# save scores
np.save('results/credit_original_scores.npy', original_scores)
np.save('results/credit_improve_scores.npy', improve_scores)
np.save('results/credit_strategy_scores.npy', strategy_scores)

# save efforts
np.save('results/credit_efforts.npy', efforts.detach().numpy())

# save groupwise scores
np.save('results/credit_original_scores_0.npy', original_scores[g==0])
np.save('results/credit_improve_scores_0.npy', improve_scores[g==0])
np.save('results/credit_strategy_scores_0.npy', strategy_scores[g==0])

np.save('results/credit_original_scores_1.npy', original_scores[g==1])
np.save('results/credit_improve_scores_1.npy', improve_scores[g==1])
np.save('results/credit_strategy_scores_1.npy', strategy_scores[g==1])

# save groupwise efforts
np.save('results/credit_efforts_0.npy', efforts.detach().numpy()[g==0])
np.save('results/credit_efforts_1.npy', efforts.detach().numpy()[g==1])


### ACSIncome

In [ ]:
c_idx = [0,1]
nc_idx = []

In [ ]:
h, f = logReg(4), logReg(4)
h.load_state_dict(torch.load('h_models/h_income_lr.pth'))
f.load_state_dict(torch.load('f_models/f_LR_income.pth'))

In [ ]:
original = pd.read_csv('data/ACSIncome_sample_raw.csv')
original['z'] = np.where(original['AGEP'] > 35, 0, 1)
X_original = original[['SCHL','WKHP','SEX','z']]
X_original = X_original.to_numpy()
X_original = torch.Tensor(X_original)

# simulate best response
We = torch.tensor([1,2], dtype = torch.float32)

efforts, x_star, x_improve = Grad_effort(h, X_original, We, c_idx, nc_idx)

In [ ]:
# sanity check
print(x_improve[1])
print(x_star[1])
print(X_original[1])
print(np.isnan(x_improve.detach().numpy()).any())
print(np.isnan(x_star.detach().numpy()).any())

In [ ]:
original_scores = np.round(h(X_original).detach().numpy(),3).reshape(-1)
improve_scores = np.round(h(x_improve).detach().numpy(),3).reshape(-1)
strategy_scores = np.round(h(x_star).detach().numpy(),3).reshape(-1)
print(original_scores.mean())
print(improve_scores.mean())
print(strategy_scores.mean())

In [ ]:
g = np.array(original['z'])

# save scores
np.save('results/income_original_scores.npy', original_scores)
np.save('results/income_improve_scores.npy', improve_scores)
np.save('results/income_strategy_scores.npy', strategy_scores)

# save efforts
np.save('results/income_efforts.npy', efforts.detach().numpy())

# save groupwise scores
np.save('results/income_original_scores_0.npy', original_scores[g==0])
np.save('results/income_improve_scores_0.npy', improve_scores[g==0])
np.save('results/income_strategy_scores_0.npy', strategy_scores[g==0])

np.save('results/income_original_scores_1.npy', original_scores[g==1])
np.save('results/income_improve_scores_1.npy', improve_scores[g==1])
np.save('results/income_strategy_scores_1.npy', strategy_scores[g==1])

# save groupwise efforts
np.save('results/income_efforts_0.npy', efforts.detach().numpy()[g==0])
np.save('results/income_efforts_1.npy', efforts.detach().numpy()[g==1])


### ACSPAP

In [ ]:
c_idx = [1,2]
nc_idx = [0]

h, f = logReg(4), logReg(4)
h.load_state_dict(torch.load('h_models/h_pap_lr.pth'))
f.load_state_dict(torch.load('f_models/f_lr_pap.pth'))

In [ ]:
original = pd.read_csv('data/ACSPAP_sample.csv')
original['z'] = np.where(original['AGEP'] > 35, 0, 1)
X_original = original[['WKHP','SCHL','PINCP','z']]
data = ACSPAP(device='cpu', decision=False)
X_original = (X_original - data.mean)/data.std_dev
X_original = X_original.to_numpy()
X_original = torch.Tensor(X_original)

# simulate best response
We = torch.tensor([1,1,2], dtype = torch.float32)

efforts, x_star, x_improve = Grad_effort(h, X_original, We, c_idx, nc_idx)

In [ ]:
# sanity check
print(x_improve[1])
print(x_star[1])
print(X_original[1])
print(np.isnan(x_improve.detach().numpy()).any())
print(np.isnan(x_star.detach().numpy()).any())

In [ ]:
original_scores = np.round(h(X_original).detach().numpy(),3).reshape(-1)
improve_scores = np.round(h(x_improve).detach().numpy(),3).reshape(-1)
strategy_scores = np.round(h(x_star).detach().numpy(),3).reshape(-1)

print(original_scores.mean())
print(improve_scores.mean())
print(strategy_scores.mean())

In [ ]:
g = np.array(original['z'])

# save scores
np.save('results/pap_original_scores.npy', original_scores)
np.save('results/pap_improve_scores.npy', improve_scores)
np.save('results/pap_strategy_scores.npy', strategy_scores)

# save efforts
np.save('results/pap_efforts.npy', efforts.detach().numpy())

# save groupwise scores
np.save('results/pap_original_scores_0.npy', original_scores[g==0])
np.save('results/pap_improve_scores_0.npy', improve_scores[g==0])
np.save('results/pap_strategy_scores_0.npy', strategy_scores[g==0])

np.save('results/pap_original_scores_1.npy', original_scores[g==1])
np.save('results/pap_improve_scores_1.npy', improve_scores[g==1])
np.save('results/pap_strategy_scores_1.npy', strategy_scores[g==1])

# save groupwise efforts
np.save('results/pap_efforts_0.npy', efforts.detach().numpy()[g==0])
np.save('results/pap_efforts_1.npy', efforts.detach().numpy()[g==1])